In [ ]:
from pytorch3d.structures import Pointclouds
from pytorch3d.vis.plotly_vis import plot_scene
from pytorch3d.renderer.cameras import PerspectiveCameras

import torch
import numpy as np
from read_write_model import read_model, qvec2rotmat

In [ ]:
# select sequence tag to show here

# tag = "apple_189_20393_38136"
tag = "teddybear_34_1479_4753"
# tag = "toytrain_240_25394_51994"

path = f"../var/colmap/{tag}/model/sparse/0"

# read COLMAP reconstruction model
cameras, images, points3D = read_model(path, ext = ".bin")

print(f"{len(cameras)} cameras found")
print(f"{len(images)} poses found")

In [ ]:
# extract point cloud
rgb = np.array([v.rgb for v in points3D.values()])
xyz = np.array([v.xyz for v in points3D.values()])

rgb = torch.tensor(rgb, dtype=torch.uint8)
xyz = torch.tensor(xyz, dtype=torch.float32)

print(f"#{rgb.shape[0]} points found")

point_cloud = Pointclouds(points=[xyz], features=[rgb/255.])

In [ ]:
# extract camera poses
focal_lengths = []
principal_points = []
R = []
T = []

for k, image in images.items():
    # intrinsics don't matter much for visualization
    focal_lengths.append((1., 1.))
    principal_points.append((0., 0.))

    # extract rotation and translation
    R.append(qvec2rotmat(image.qvec))
    T.append(image.tvec)

cams = PerspectiveCameras(
    focal_length=focal_lengths,
    principal_point=principal_points,
    R=R,
    T=T,
)

In [ ]:
plot_scene(
    {"COLMAP Reconstruction (using SiLK keypoints)": {tag: point_cloud[0], "cameras": cams}},
    pointcloud_max_points=rgb.shape[0],
    raybundle_max_rays=rgb.shape[0],
    raybundle_max_points_per_ray=rgb.shape[0],
    width=400,
    height=400,
)